In [ ]:
import cv2
import numpy as np

def blind_deconvolution(blurry, psf_guess, iterations=50):

  restored = np.copy(blurry)

  for _ in range(iterations):

    # Use np.divide for safety
    deconvolved = np.divide(restored, psf_guess + 1e-8)
    
    blurred = cv2.filter2D(deconvolved, -1, psf_guess)
    error = blurry - blurred
    
    updated_psf = cv2.filter2D(error, -1, deconvolved)
    psf_guess = psf_guess + 0.2 * updated_psf
    
    restored = deconvolved + 0.2 * cv2.filter2D(error, -1, updated_psf)

  return restored, psf_guess


# Load blurred image
blurry = cv2.imread('blurred_image.jpg', 0)  

# Initial PSF guess
psf_guess = cv2.GaussianBlur(np.ones(shape=(15,15)), (15,15), 0)

# Ensure PSF guess matches image shape
h, w = blurry.shape
psf_guess = cv2.resize(psf_guess, (w, h))

# Perform blind deconvolution
restored, psf = blind_deconvolution(blurry, psf_guess, iterations=100)

# Save outputs
cv2.imwrite('restored.jpg', restored)
cv2.imwrite('psf.jpg', psf)